In [1]:
import pandas as pd
import json
import ast
!pip install fastparquet

# Leitura do arquivo TXT - Utilizando literal_eval para transformações de string para objetos

In [2]:
import ast
  

with open('export_deals.txt',encoding="utf-8") as f:
    data = f.read()

d = ast.literal_eval(data)
  

# Função para organização das hierarquias da estrutura dos dados 

A função conct_dict, foi utilizada para definir a hierarquia dos dados considerando a estrutura de dados no formato dicionário: pois a estrutura correspondente a um registro era representado por um dicionário. No entanto, os dicionários contiam outras estruturas de dados como: dicinários e listas. Está função tem como objetivo extrair os valores destas estruturas aninhadas e padronizar em um dado comum, para isso foi utilizado o nome da chave externa concatenado com nome da chave do interna do dicionário. Com a padronização herdada da modelagem de banco de dados.

Por exemplo: A a estrutura:
{'id': 5373, 'creator_user_id': {'id': 4571337, 'name': 'Bruno Martins', 'email': 'bruno.martins@hsb.com.br'...
Foi transformado para:
{id': 5373, 'creator_user_id_id': 4571337, 'creator_user_id_name': 'Bruno Martins', 'creator_user_id_email': 'bruno.martins@hsb.com.br' ..

In [4]:
def conct_dict (i,dict,name):
    list_name_key =list(d[i][name].keys())
    val_conc = []
    list_values = []
    for key_in_list in list_name_key:
        val_conc.append(f"{name}_{key_in_list}")
    list_values = list(d[i][name].values())
    res = {val_conc[i]: list_values[i] for i in range(len(val_conc))}
    return res 


Para resolução das listas aninhadas contidas nos dicionários a função abaixo foi utilizada para extração dos dados, neste sample de dados foi observado que as listas contiam dicionários e a extração também foi realizada. 

In [5]:
def list_conct_dict (i,dict,name,pos):
    list_name_key =list(d[i][name][pos].keys())
    val_conc = []
    list_values = []
    for key_in_list in list_name_key:
        val_conc.append(f"{name}_{key_in_list}")
    list_values = list(d[i][name][pos].values())
    res = {val_conc[i]: list_values[i] for i in range(len(val_conc))}
    return res 

# Tratamento dos dicionários mais internos

Para extrair os dados foi necessário percorrer cada amostra do conjunto, para verificação da extração foi verificado o tipo de cada chave do dicionário. E se o tipo de dados da chave correspondese a um outro dicionário é realizado a extração dos valores do dicionário mais interno.

In [6]:
#Tratando dicionário
from tqdm import tqdm
lista_aux_del = []
for i in tqdm(range(len(d)),total=len(d)):

    for j in d[i].keys():
        new_values_key = {} 
        if type(d[i][j]) is dict:
            new_values_key = conct_dict(i, d[i][j], j)
            d[i][j] = new_values_key
            lista_aux_del.append(j)

100%|██████████| 1000/1000 [00:00<00:00, 25708.90it/s]
